# Set Up


In [1]:
from google.colab import drive
import zipfile
import os

drive.mount('/content/drive')

# Get Dataset
with zipfile.ZipFile("/content/drive/MyDrive/ADNI_SLICE_TRAINING_DATA.zip", 'r') as zip_ref:
    zip_ref.extractall("/content/dataset")


Mounted at /content/drive


# Imports

In [2]:
import sys
import tensorflow as tf
from keras.preprocessing import image_dataset_from_directory
from tensorflow.keras.applications.inception_v3 import InceptionV3
from datetime import datetime
import os

# Config


In [3]:
STRATEGY = tf.distribute.get_strategy()
AUTOTUNE = tf.data.AUTOTUNE

BATCH_SIZE = 16 * STRATEGY.num_replicas_in_sync
EPOCHS = 10

IMAGE_SIZE = [200, 200]

MODEL_SAVE_PATH = "/content/drive/MyDrive/models"

DATASET_PATH = "/content/dataset/ADNI_SLICE_TRAINING_DATA"

MODEL_NAME = "SLICE_EXTRACTOR_DENSE_NET"

# Model

In [4]:
base_dir = f"{MODEL_SAVE_PATH}/{MODEL_NAME}-{datetime.now().strftime('%Y-%m-%d-%H:%M')}"
os.makedirs(base_dir, exist_ok=True)


print("REPLICAS: ", STRATEGY.num_replicas_in_sync)

train_ds = image_dataset_from_directory(
    DATASET_PATH,
    labels="inferred",
    label_mode="binary",
    image_size=IMAGE_SIZE,
    validation_split=0.2,
    subset="training",
    seed=1337
).prefetch(buffer_size=AUTOTUNE)

validation_ds = image_dataset_from_directory(
    DATASET_PATH,
    labels="inferred",
    label_mode="binary",
    image_size=IMAGE_SIZE,
    validation_split=0.2,
    subset="validation",
    seed=1337
).prefetch(buffer_size=AUTOTUNE)

with STRATEGY.scope():
    base_model = InceptionV3(input_shape=(
        *IMAGE_SIZE, 3), include_top=False, weights='imagenet')

    for layer in base_model.layers:
        layer.trainable = False

    x = tf.keras.layers.Flatten()(base_model.output)
    x = tf.keras.layers.Dense(1024, activation='relu')(x)
    x = tf.keras.layers.Dropout(0.2)(x)

    # Add a final sigmoid layer with 1 node for classification output
    x = tf.keras.layers.Dense(1, activation='sigmoid')(x)

    model = tf.keras.models.Model(base_model.input, x)

model.summary()

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy'])

# Defining Callbacks
save_best = tf.keras.callbacks.ModelCheckpoint(
    filepath=f"{base_dir}/model.h5", monitor='val_loss', save_best_only=True)
reduce_lr = tf.keras.callbacks.ReduceLROnPlateau(
    patience=10, monitor='val_accuracy', factor=0.6, min_lr=0.0000001)

history = model.fit(
    train_ds,
    validation_data=validation_ds,
    epochs=EPOCHS,
    callbacks=[save_best, reduce_lr]
)


REPLICAS:  1
Found 2000 files belonging to 2 classes.
Using 1600 files for training.
Found 2000 files belonging to 2 classes.
Using 400 files for validation.
87910968/87910968 [==============================] - 1s 0us/step
Model: "model"
__________________________________________________________________________________________________
 Layer (type)                Output Shape                 Param #   Connected to                  
 input_1 (InputLayer)        [(None, 200, 200, 3)]        0         []                            
                                                                                                  
 conv2d (Conv2D)             (None, 99, 99, 32)           864       ['input_1[0][0]']             
                                                                                                  
 batch_normalization (Batch  (None, 99, 99, 32)           96        ['conv2d[0][0]']              
 Normalization)                                                      

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


50/50 [==============================] - 227s 4s/step - loss: 89.0050 - accuracy: 0.8100 - val_loss: 6.4749 - val_accuracy: 0.9325 - lr: 0.0010
Epoch 2/10
50/50 [==============================] - 217s 4s/step - loss: 3.8882 - accuracy: 0.9406 - val_loss: 2.6888 - val_accuracy: 0.9525 - lr: 0.0010
Epoch 3/10
50/50 [==============================] - 221s 4s/step - loss: 2.1080 - accuracy: 0.9569 - val_loss: 2.8598 - val_accuracy: 0.9325 - lr: 0.0010
Epoch 4/10
50/50 [==============================] - 215s 4s/step - loss: 0.9992 - accuracy: 0.9688 - val_loss: 0.9845 - val_accuracy: 0.9675 - lr: 0.0010
Epoch 5/10
50/50 [==============================] - 218s 4s/step - loss: 0.8911 - accuracy: 0.9644 - val_loss: 1.4900 - val_accuracy: 0.9600 - lr: 0.0010
Epoch 6/10
50/50 [==============================] - 204s 4s/step - loss: 1.1272 - accuracy: 0.9544 - val_loss: 1.2032 - val_accuracy: 0.9575 - lr: 0.0010
Epoch 7/10
50/50 [==============================] - 222s 4s/step - loss: 0.4675 - accu